# Orders
Orders are instructions to broker to purchase or sell something under certain conditions. Currently supported orders are Market, Limit, Take Profit, Take Profit Limit, Stop Loss and Stop Loss Limit. This notebook explains how to submit and cancel different types of orders. Actual run of the strategy is omitted for simplicity.

**NOTE**:
- Unlike other notebooks, content of this notebook only shows syntax and can't be executed for meaningful results (as we don't run any actual backtesting here)
- BUY and SELL often submitted simultaneously here just for simplicity. You can't expect to have enough balance to post one more order once the first one has already took the full balance as its margin requirement.

## How to submit a Market Order

Using the shortcut

In [8]:
from backintime import FuturesStrategy
class DummyStrategy(FuturesStrategy):
    def tick(self):
        market_buy = self.buy()   # That's it! Market order is posted for the full amount of funds available for trading
        market_sell = self.sell() # The same for sell
        '''It is not necessarily to post orders for the full amount. 
        You can specify any number as `amount` and the order will be submitted
        if there are enough funds available on your balance.'''

Using the `FuturesBroker` API directly (more verbose)

In [7]:
from backintime import FuturesStrategy
from backintime.broker import OrderSide, MarketOrderOptions
class DummyStrategy(FuturesStrategy):
    def tick(self):
        max_buy = self.broker.max_funds_for_futures
        max_sell = self.broker.balance.available_contracts   # or self.position, it is the same

        buy_options = MarketOrderOptions(OrderSide.BUY, amount=max_buy)
        sell_options = MarketOrderOptions(OrderSide.SELL, amount=max_sell)

        market_buy = self.broker.submit_market_order(buy_options)
        market_sell = self.broker.submit_market_order(sell_options)
        '''It is not necessarily to post orders for the full amount. 
        You can specify any number as `amount` and the order will be submitted
        if there are enough funds available on your balance.'''

## How to submit a Limit Order

Using the shortcut

In [ ]:
from decimal import Decimal
from backintime import FuturesStrategy
class DummyStrategy(FuturesStrategy):
    def tick(self):
        limit_buy = self.limit_buy(order_price=Decimal('100'))   # Much like with Market Orders.
        limit_sell = self.limit_sell(order_price=Decimal('100')) # The same for sell
        '''It is not necessarily to post orders for the full amount. 
        You can specify any number as `amount` and the order will be submitted
        if there are enough funds available on your balance.'''

Using the `FuturesBroker` API directly (more verbose)

In [9]:
from decimal import Decimal
from backintime import FuturesStrategy
from backintime.broker import OrderSide, LimitOrderOptions
class DummyStrategy(FuturesStrategy):
    def tick(self):
        max_buy = self.broker.max_funds_for_futures
        max_sell = self.broker.balance.available_contracts   # or self.position, it is the same

        buy_options = LimitOrderOptions(OrderSide.BUY, order_price=Decimal(100), amount=max_buy)
        sell_options = LimitOrderOptions(OrderSide.SELL, order_price=Decimal(100), amount=max_sell)

        limit_buy = self.broker.submit_limit_order(buy_options)
        limit_sell = self.broker.submit_limit_order(sell_options)
        '''It is not necessarily to post orders for the full amount. 
        You can specify any number as `amount` and the order will be submitted
        if there are enough funds available on your balance.'''

## How to submit a Limit Order with TP/SL

Using the shortcut

In [12]:
from decimal import Decimal
from backintime import FuturesStrategy
from backintime.broker import TakeProfitOptions, StopLossOptions
class DummyStrategy(FuturesStrategy):
    def tick(self):
        # NOTE: no need to specify order side, it is inferred from the parent Limit order (would be inverted)
        tp_sell_options = TakeProfitOptions(trigger_price=Decimal(110), percentage_amount=Decimal(100))
        sl_sell_options = StopLossOptions(trigger_price=Decimal(90), percentage_amount=Decimal(100))
        # NOTE: no need to specify order side, it is inferred from the parent Limit order (would be inverted)
        tp_buy_options = TakeProfitOptions(trigger_price=Decimal(90), percentage_amount=Decimal(100))
        sl_buy_options = StopLossOptions(trigger_price=Decimal(110), percentage_amount=Decimal(100))

        limit_buy = self.limit_buy(order_price=Decimal('100'), 
                                   take_profit=tp_sell_options,
                                   stop_loss=sl_sell_options)
         
        limit_sell = self.limit_sell(order_price=Decimal('100'),
                                    take_profit=tp_buy_options,
                                    stop_loss=sl_buy_options)

Using the `FuturesBroker` API directly (more verbose)

In [13]:
from decimal import Decimal
from backintime import FuturesStrategy
from backintime.broker import LimitOrderOptions, TakeProfitOptions, StopLossOptions
class DummyStrategy(FuturesStrategy):
    def tick(self):
        # NOTE: no need to specify order side, it is inferred from the parent Limit order (would be inverted)
        tp_sell_options = TakeProfitOptions(trigger_price=Decimal(110), percentage_amount=Decimal(100))
        sl_sell_options = StopLossOptions(trigger_price=Decimal(90), percentage_amount=Decimal(100))
        # NOTE: no need to specify order side, it is inferred from the parent Limit order (would be inverted)
        tp_buy_options = TakeProfitOptions(trigger_price=Decimal(90), percentage_amount=Decimal(100))
        sl_buy_options = StopLossOptions(trigger_price=Decimal(110), percentage_amount=Decimal(100))

        max_buy = self.broker.max_funds_for_futures
        max_sell = self.broker.balance.available_contracts   # or self.position, it is the same

        buy_options = LimitOrderOptions(OrderSide.BUY, 
                                        order_price=Decimal(100), amount=max_buy,
                                        take_profit=tp_sell_options,
                                        stop_loss=sl_sell_options)

        sell_options = LimitOrderOptions(OrderSide.SELL, 
                                         order_price=Decimal(100), amount=max_sell,
                                         take_profit=tp_buy_options,
                                         stop_loss=sl_buy_options)

        limit_buy = self.broker.submit_limit_order(buy_options)
        limit_sell = self.broker.submit_limit_order(sell_options)

## How to open Short position

In [ ]:
from decimal import Decimal
from backintime import FuturesStrategy
class DummyStrategy(FuturesStrategy):
    def tick(self):
        market_short = self.market_short()   # Open Short position for the full amount available

## Going Short with TP/SL
Note that it is not required to use TP/SL to close Short position. Any BUY order can be used

In [3]:
from decimal import Decimal
from backintime import FuturesStrategy

class DummyStrategy(FuturesStrategy):
    def tick(self):
        tp_options = TakeProfitOptions(percentage_amount=Decimal('100'), trigger_price=Decimal('900'))
        sl_options = StopLossOptions(percentage_amount=Decimal('100'), trigger_price=Decimal('1100'))
        short = self.limit_short(order_price=Decimal('1000'), 
                                 take_profit=tp_options, 
                                 stop_loss=sl_options)

## How to cancel order

In [14]:
from decimal import Decimal
from backintime import FuturesStrategy
class DummyStrategy(FuturesStrategy):
    def tick(self):
        limit_buy = self.limit_buy(order_price=Decimal('100'))
        self.broker.cancel_order(limit_buy.order_id)

Note that if the order has linked TP/SL they should be cancelled additionally. That's not the case if their parent Limit has been cancelled (since no TP/SL were posted yet). But if the Limit order is executed already, keep in mind that 1) it can't be cancelled anymore 2) if you need to cancel its TP/SL you need to invoke `cancel_order` for each 

## How to check if order is executed/cancelled
You need to check its status

In [1]:
from decimal import Decimal
from backintime import FuturesStrategy
from backintime.broker.base import OrderStatus
class DummyStrategy(FuturesStrategy):
    def tick(self):
        # Of course it won't be executed or cancelled right away but this is just an example
        limit_buy = self.limit_buy(order_price=Decimal('100'))
        if limit_buy.status is OrderStatus.EXECUTED:
            pass   # It is executed
        elif limit_buy.status is OrderStatus.CANCELLED:
            pass   # It is cancelled

Note that for TP/SL orders there's also `OrderStatus.SYS_CANCELLED` status. It is set if an order was cancelled by broker (e.g. when TP is executed, SL gets cancelled).

## How to check status of linked TP/SL orders

In [2]:
from decimal import Decimal
from backintime import FuturesStrategy
from backintime.broker import TakeProfitOptions, StopLossOptions

class DummyStrategy(FuturesStrategy):
    def tick(self):
        tp_sell_options = TakeProfitOptions(trigger_price=Decimal(110), percentage_amount=Decimal(100))
        sl_sell_options = StopLossOptions(trigger_price=Decimal(90), percentage_amount=Decimal(100))

        limit_buy = self.limit_buy(order_price=Decimal('100'), 
                                   take_profit=tp_sell_options,
                                   stop_loss=sl_sell_options)

        '''At this point `.take_profit` is None since it has not been posted yet
        But after the Limit BUY execution you will be able to see its TP.'''
        if limit_buy.take_profit:
            print(limit_buy.take_profit.status)

        '''At this point `.stop_loss` is None since it has not been posted yet
        But after the Limit BUY execution you will be able to see its SL.'''
        if limit_buy.stop_loss:
            print(limit_buy.stop_loss.status)
